In [3]:
import pandas as pd
import numpy as np

# =======================================================
# 1. Load Data
# =======================================================
print("Loading GSE33000 data...")

data = pd.read_csv(
    r"N:\GSE33000_raw_data.txt\GSE33000_raw_data.txt", 
    sep="\t", 
    low_memory=False
)

print(f"Original shape: {data.shape}")
print(f"Total probes: {len(data)}\n")

# =======================================================
# 2. Clean Gene Names
# =======================================================
data["Gene_Clean"] = data["Gene"].str.split(",").str[0]
data = data[data["Gene_Clean"].notna()]
data = data[~data["Gene_Clean"].str.contains("RSE_|LOC", na=False)]

print(f"After cleaning gene names: {len(data)} probes\n")

# =======================================================
# 3. Identify Sample Columns
# =======================================================
metadata_cols = ["reporterID", "Gene", "Transcript", "Gene_Clean"]
sample_cols = [c for c in data.columns if c not in metadata_cols]

print(f"Number of samples: {len(sample_cols)}")
print("First 5 sample columns:", sample_cols[:5], "\n")

# Convert expression values to numeric
expression = data[sample_cols].apply(pd.to_numeric, errors="coerce")

# Warn if others exist
nan_count = expression.isna().sum().sum()
if nan_count > 0:
    print(f"⚠️ Found {nan_count} non-numeric values (converted to NaN)\n")

# =======================================================
# 4. Compute Stats Per Gene
# =======================================================
data["variance"] = expression.var(axis=1)
data["mean_expression"] = expression.mean(axis=1)
data["std_dev"] = expression.std(axis=1)

# Remove any rows where stats couldn't be calculated
data = data[data["variance"].notna()]
print(f"After removing invalid rows: {len(data)} probes\n")

# =======================================================
# 5. Filter Low Expression
# =======================================================
filtered = data[data["mean_expression"] > 0.1]
print(f"After filtering low expression: {len(filtered)} probes\n")

# =======================================================
# 6. Rank Genes by Variance
# =======================================================
ranked = filtered.sort_values("variance", ascending=False)

# Remove duplicate genes (keep highest variance)
unique_genes = ranked.drop_duplicates(subset="Gene_Clean", keep="first")
print(f"Unique high-variance genes: {len(unique_genes)}\n")

# Take top 200
top_200 = unique_genes.head(200)

# =======================================================
# 7. Prepare Output
# =======================================================
output = top_200[[
    "Gene_Clean",
    "Gene",
    "variance",
    "mean_expression",
    "std_dev"
]].copy()

output.columns = [
    "gene_symbol",
    "original_gene_field",
    "variance",
    "mean_expression",
    "std_dev"
]

output["evidence_type"] = "E"
output["source"] = "GSE33000"

output.to_csv("high_variance_genes_gse33000.csv", index=False)

# =======================================================
# Summary
# =======================================================
print("="*60)
print("✓ COMPLETE!")
print("="*60)
print(f"\nSaved {len(output)} high variance genes\n")

print("📊 Top 20 high-variance genes:")
print("="*60)
print(output[["gene_symbol", "variance", "mean_expression"]].head(20).to_string(index=False))

print("\n📈 STATISTICS:")
print("="*60)
print(f"Variance range: {output['variance'].min():.6f} - {output['variance'].max():.6f}")
print(f"Mean expression range: {output['mean_expression'].min():.4f} - {output['mean_expression'].max():.4f}")

# =======================================================
# 8. Check for Known AD Genes
# =======================================================
ad_genes = [
    "APOE","APP","PSEN1","PSEN2","TREM2","CLU","BIN1","CD33",
    "ABCA7","CR1","PICALM","MS4A6A","CD2AP","EPHA1","INPP5D"
]

found_ad = output[output["gene_symbol"].isin(ad_genes)]

print("\n🧬 Known AD genes found:", len(found_ad))
print("="*60)

if not found_ad.empty:
    print(found_ad[["gene_symbol", "variance", "mean_expression"]].to_string(index=False))
else:
    print("None found in top 200, checking top 500...")
    top_500 = unique_genes.head(500)
    found_ad_500 = top_500[top_500["Gene_Clean"].isin(ad_genes)]

    if not found_ad_500.empty:
        print(f"Found {len(found_ad_500)} in top 500:")
        print(found_ad_500[["Gene_Clean", "variance"]].head(15).to_string(index=False))
    else:
        print("None found in top 500 either.")


Loading GSE33000 data...
Original shape: (39280, 1251)
Total probes: 39280

After cleaning gene names: 35197 probes

Number of samples: 1248
First 5 sample columns: ['HBTRC_PF_Pool_1', 'HBTRC_PF_Pool_2', 'HBTRC_PF_Pool_3', 'HBTRC_PF_Pool_4', 'HBTRC_PF_Pool_5'] 

⚠️ Found 555880 non-numeric values (converted to NaN)

After removing invalid rows: 35197 probes

After filtering low expression: 24142 probes

Unique high-variance genes: 19294

✓ COMPLETE!

Saved 200 high variance genes

📊 Top 20 high-variance genes:
      gene_symbol     variance  mean_expression
             COX2 15028.519205       169.874097
             CYTB 13589.435657       150.536070
              ND3 13390.865831       159.764876
              ND1 13202.928669       155.531812
Contig5109_RC_sat 11655.267492       179.662579
              ND2 10933.643957       123.394293
           MTND4L 10391.301126       139.111669
         FLJ25778  7661.631369       120.626090
            GFRA3  5992.397597       106.826278
    

In [5]:
import requests
import time
import pandas as pd

def get_entrez_id(gene_symbol):
    """Return Entrez Gene ID for a given human gene symbol."""
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "gene",
        "term": f"{gene_symbol}[Gene Name] AND human[Organism]",
        "retmode": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        ids = data.get("esearchresult", {}).get("idlist", [])
        return ids[0] if ids else None
    except:
        return None


# ============================
# 1. Load genes
# ============================
genes = pd.read_csv("high_variance_genes_gse33000.csv")

# ============================
# 2. Get Entrez IDs
# ============================
entrez_list = []

for gene in genes["gene_symbol"]:
    eid = get_entrez_id(gene)
    entrez_list.append(eid)
    time.sleep(0.3)   # be gentle to NCBI


# ============================
# 3. Add & Save Output
# ============================
genes["entrez_id"] = entrez_list

genes.to_csv("genes_with_entrez_ids.csv", index=False)

print("Done! Saved → genes_with_entrez_ids.csv")


Done! Saved → genes_with_entrez_ids.csv
